In [1]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/preprocessed_data.csv'

In [4]:
preprocessed_data = pd.read_csv(path)

In [5]:
preprocessed_data.head()

,comment_text,Score
0,explanation edits made username hardcore metal...,0
1,aww match background colour seemingly stuck th...,0
2,hey man really trying edit war guy constantly ...,0
3,make real suggestion improvement wondered sect...,0
4,sir hero chance remember page,0


In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.7 MB/s eta 0:00:00


In [7]:
from transformers import DistilBertTokenizerFast
from transformers import  TFDistilBertForSequenceClassification
import numpy as np

In [8]:
import tensorflow as tf

In [49]:
preprocessed_data.dropna(inplace=True)

In [29]:
comments = preprocessed_data['comment_text'].values.tolist()
labels = preprocessed_data['Score'].values.tolist()

In [30]:
from sklearn.model_selection import train_test_split
training_sentences , validation_sentences , training_labels , validation_labels = train_test_split(comments,labels,test_size=0.2)

In [31]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [32]:
tokenizer([training_sentences[0]],truncation=True,padding=True,max_length=128)

{'input_ids': [[101, 2307, 2804, 17850, 2080, 11721, 2099, 6606, 11721, 18505, 2517, 5896, 9046, 2544, 2022, 5004, 21007, 4872, 3352, 2172, 5720, 3185, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [35]:
train_encodings=tokenizer(training_sentences,truncation=True,padding=True)
val_encodings=tokenizer(validation_sentences,truncation=True,padding=True)

In [45]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), training_labels))


In [38]:
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), validation_labels))


In [40]:
model=TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [56]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5,epsilon=1e-8)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

In [57]:
test_sentence = 'think fagget get oife burn hell hate sorry cant sex running conndoms'

In [59]:
predict_input=tokenizer.encode(test_sentence,truncation=True,padding=True,return_tensors='tf')

In [61]:
tf_output=model.predict(predict_input)[0]
tf_prediction= tf.nn.softmax(tf_output,axis=1)
labels=['Okay','Alert']
label=tf.argmax(tf_prediction,axis=1)
label=label.numpy()
print(labels[label[0]])

1/1 [==============================] - 2s 2s/step
Alert


In [63]:
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/2


AttributeError: ignored